# Wikipedia Notable Deaths 

## [Notebook 1 of 4:   Data Collection](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wikipedia_notable_deaths_data_collection_thanak_2022_06_10.ipynb)

## Context

This case study was chosen by this author as an end-to-end portfolio project for the following reasons:  
1. accessibility of the dataset
2. potential for use and demonstration of a wide range of skills, including coding in Python (using Pycharm and Jupyter Notebooks), version control (using Git, GitHub, and ReviewNB), web scraping (using Scrapy), relational database management (using SQLite and [SQLite Viewer](https://inloop.github.io/sqlite-viewer/)),  data cleaning, natural language processing (NLP using regular expressions and fuzzywuzzy), Exploratory Data Analysis (EDA using numpy, pandas, matplotlib, plotly, and seaborn), data preprocessing, unsupervised learning (Kmeans and hierachical clustering), supervised learning (regression models), user interface for predictions.
3. in particular, a dataset with potential for regression modeling was chosen to gain more exposure to its associated algorithms as this author had relative broader experience with classification models in past projects.
    

## Objective

The...

## Data Dictionary
Variable: Description

## Data Collection
- Data was collected from 6/9/22 to 6/10/22, using Scrapy. 

### 6/9/2022

- The [Wikipedia](https://en.wikipedia.org/wiki/Main_Page) page, [List of Deaths by Year](https://en.wikipedia.org/wiki/Lists_of_deaths_by_year), contains entries for as early as 1987, to the present day.  
- 1994 was chosen as the start year for collection as it is the first year with entries following the current format: "Name, age, country of citizenship at birth, subsequent country of citizenship (if applicable), reason for notability, cause of death (if known), and reference."
- For ease of pagination, [Deaths in January 1994](https://en.wikipedia.org/wiki/Deaths_in_January_1994) was the start url for scraping, proceeding month by month through subsequent pages.
- Spider ["by_year"](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wikipedia_notable_deaths/spiders/by_year.py) scraped `month_year`, `day`, `name`, `info` ("age, country of citizenship at birth, subsequent country of citizenship (if applicable), reason for notability, and cause of death (if known)"), and `link` for each entry on each month's page.  
- The project's [pipelines.py](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wikipedia_notable_deaths/pipelines.py)* wrote results to SQLite table wp_deaths_94_to_22 within [wp_deaths_94_to_22.db](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wp_deaths_94_to_22.db).  This scraping was successful for January, 1994 through May, 2022 data.
- [Deaths in 2022 -- June](https://en.wikipedia.org/wiki/Deaths_in_2022#June)--the current month's page--varied in format, and was therefore scraped separately, after `num_references` was scraped for the previous entries.
- The original order of entries was preserved by Spider ["by_year"](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wikipedia_notable_deaths/spiders/by_year.py) in [wp_deaths_94_to_22.db](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wp_deaths_94_to_22.db).  Scrapy trades pagination order for speed, which is noticable when pagination is of higher magnitude.  Therefore, scraping each entry's page for number of references was done separately, as the order was sure to vary. Spider ["references"](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wikipedia_notable_deaths/spiders/references.py) scraped for number of references.  The project's [pipelines.py](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wikipedia_notable_deaths/pipelines.py)* wrote results to SQLite table wp_reference_counts in [wp_reference_counts.db](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wp_reference_counts_2.db), with the results falling ~13,000 rows short of the 133,769 rows captured by Spider ["by_year"](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wikipedia_notable_deaths/spiders/by_year.py).
- Finally, the [June, 2022](https://en.wikipedia.org/wiki/Deaths_in_2022#June) page was scraped by Spider ["June_2022"](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wikipedia_notable_deaths/spiders/June_2022.py), successfully capturing all of the previous fields, including number of references.  Note that the number of pages was some order of magnitude smaller.
- The project's [pipelines.py](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wikipedia_notable_deaths/pipelines.py)* wrote results to SQLite table wp_deaths_June_2022 in [wp_deaths_June_2022.db](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wp_deaths_June_2022.db), resulting in 145 rows from the first part of June, 2022.

### 6/10/2022
The remaining data collection steps are outlined in this notebook:
1. [Reading, sampling, and checking data shape](#step1)
    - SQLite tables wp_deaths_94_to_22, wp_reference_counts, and wp_June_2022 were read in as pandas dataframes.
2. [Combining dataframes](#step2)
    - Dataframes for wp_deaths_94_to_22 and wp_reference_counts were combined using `link` as the unique identifier.
    - Dataframe for wp_deaths_June_2022 was added.
3. [Duplicate Rows](#step3)
    - 9 rows of duplicate entries were dropped.
4. [Missing Values](#step4)
    - 5 rows missing essential data were dropped.
    - 6 entries had missing `name` and `num_references`, but contained the name in the `info` feature, to be extracted later during data cleaning.  As these entries had no associated page, their `num_references` was set equal to to 0.
5. [Missing `num_references` Values](#step5)
    - A single modification was made to the original XPath for the original "by_year" Spider, to match a variation on the pages for links with missing `num_references`.  
    - Those pages were then were rescraped iteratively by Spiders ["refs2"](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wikipedia_notable_deaths/spiders/refs2.py), ["refs3"](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wikipedia_notable_deaths/spiders/refs3.py), ["refs4"](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wikipedia_notable_deaths/spiders/refs4.py), and ["refs5"](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wikipedia_notable_deaths/spiders/refs5.py).
    - The project's [pipelines.py](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wikipedia_notable_deaths/pipelines.py)* wrote their respective results to SQLite tables refs2 in [refs2.db](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/refs2.db), refs3 in [refs3.db](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/refs3.db), refs4 in [refs4.db](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/refs4.db), and refs5 in [refs5.db](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/refs5.db).
    - During each iteration, the additional `num_references` were added to the main dataset and remaining links with missing `num_references` values were identified.
    - After the final iteration, 92 entries remained with missing `num_references` that were all missing an associated page, so `num_references` was set equal to to 0.  These entries contained all of the other relevant features, so were preserved.
    - The resultant raw dataset has 133,900 rows and 6 columns.

\*  The current version of [pipelines.py](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wikipedia_notable_deaths/pipelines.py) reflects its use by the most recent project spider crawled, as it is reused for all [spiders](https://github.com/teresahanak/wikipedia-notable-deaths/tree/main/wikipedia_notable_deaths/spiders) within the [Scrapy project folder](https://github.com/teresahanak/wikipedia-notable-deaths/tree/main/wikipedia_notable_deaths).

## Importing necessary libraries

In [1]:
# To structure code automatically
%load_ext nb_black

# To import sqlite databases
import sqlite3 as sql

# To help with reading and manipulating data
import pandas as pd

# To define maximum number of columns to be displayed in a dataframe
pd.set_option("display.max_columns", None)
# To define the maximum number of rows to be displayed in a dataframe
pd.set_option("display.max_rows", 200)

# To supress scientific notations for a dataframe
pd.set_option("display.float_format", lambda x: "%.3f" % x)

# To supress warnings
# import warnings

# warnings.filterwarnings("ignore")

# To set some visualization attributes
pd.set_option("max_colwidth", 150)

<IPython.core.display.Javascript object>

## Data Overview

<a id='step1'></a>
### Reading, sampling, and checking data shape

### January 1994 through May 2022 Data (without reference counts)

In [2]:
# Reading the wp_deaths_94_to_22 dataset from sql db and table
conn = sql.connect("wp_deaths_94_to_22.db")
raw_94_to_22 = pd.read_sql("SELECT * FROM wp_deaths_94_to_22", conn)

# Making a working copy
df_94_to_22 = raw_94_to_22.copy()

# Checking the shape
print(f"There are {df_94_to_22.shape[0]} rows and {df_94_to_22.shape[1]} columns.")

# Checking first 2 rows of the data
df_94_to_22.head(2)

There are 133769 rows and 5 columns.


,month_year,day,name,info,link
0,January 1994,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer)
1,January 1994,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty


<IPython.core.display.Javascript object>

In [3]:
# Checking last 2 rows of the data
df_94_to_22.tail(2)

,month_year,day,name,info,link
133767,May 2022,31,Dave Smith,", 72, American sound engineer, founder of Sequential.",https://en.wikipedia.org/wiki/Dave_Smith_(engineer)
133768,May 2022,31,Wang Zherong,", 86, Chinese tank designer, member of the Chinese Academy of Engineering.",https://en.wikipedia.org/wiki/Wang_Zherong


<IPython.core.display.Javascript object>

In [4]:
# Checking a sample of the data
df_94_to_22.sample(5)

,month_year,day,name,info,link
127470,October 2021,5,Pierre Légaré,", 72, Canadian comedian.",https://en.wikipedia.org/wiki/Pierre_L%C3%A9gar%C3%A9
37879,August 2008,3,Aurelius H. Piper Sr.,", 92, American hereditary chief of the Golden Hill Paugussett Indian Nation since 1959, natural causes.",https://en.wikipedia.org/wiki/Aurelius_H._Piper_Sr.
6604,August 1996,8,Joseph Asajirō Satowaki,", 92, Japanese Catholic Bishop.",https://en.wikipedia.org/wiki/Joseph_Asajiro_Satowaki
65336,November 2013,2,Brita Borge,", 82, Norwegian politician.",https://en.wikipedia.org/wiki/Brita_Borge
99264,September 2018,18,Robert Venturi,", 93, American architect, Pritzker Prize winner (1991), complications from Alzheimer's disease.",https://en.wikipedia.org/wiki/Robert_Venturi


<IPython.core.display.Javascript object>

#### Observations:
- There are 133,769 rows and 5 columns in the data from January, 1994 through May, 2022.
- The number of references was scraped separately.

### January 1994 through May 2022 Reference Count Data

In [5]:
# Reading the wp_reference_counts_2 dataset from sql db and table
conn = sql.connect("wp_reference_counts_2.db")
raw_reference_counts = pd.read_sql("SELECT * FROM wp_reference_counts_2", conn)

# Making a working copy
df_reference_counts = raw_reference_counts.copy()

# Checking the shape
print(
    f"There are {df_reference_counts.shape[0]} rows and {df_reference_counts.shape[1]} columns."
)

# Checking first 2 rows of the data
df_reference_counts.head(2)

There are 120368 rows and 2 columns.


,link,num_references
0,https://en.wikipedia.org/wiki/Lys_Gauty,5
1,https://en.wikipedia.org/wiki/William_Chappell_(dancer),21


<IPython.core.display.Javascript object>

In [6]:
# Checking last 2 rows of the data
df_reference_counts.tail(2)

,link,num_references
120366,https://en.wikipedia.org/wiki/Shirley_Thomas_(USC_professor),6
120367,https://en.wikipedia.org/wiki/James_Doohan,52


<IPython.core.display.Javascript object>

In [7]:
# Checking a sample of the data
df_reference_counts.sample(5)

,link,num_references
99331,https://en.wikipedia.org/wiki/Stephen_Greene_(artist),2
105250,https://en.wikipedia.org/wiki/Peter_Barbour,11
73348,https://en.wikipedia.org/wiki/Sinake_Giregire,13
101432,https://en.wikipedia.org/wiki/Jonathan_Mann_(WHO_official),11
25697,https://en.wikipedia.org/wiki/Kanybek_Isakov,5


<IPython.core.display.Javascript object>

#### Observations:
- Here, we see that there are ~13,000 fewer rows for the reference data, indicating some pages were not successfully scraped to obtain the number of references for the individual.
- After combining the three dataframes, we can examine those pages and reattempt scraping them, in order to obtain the missing information.

### June 2022 Data

In [8]:
# Reading the wp_deaths_June_2022 dataset from sql db and table
conn = sql.connect("wp_deaths_June_2022.db")
raw_June_2022 = pd.read_sql("SELECT * FROM wp_deaths_June_2022", conn)

# Making a working copy
df_June_2022 = raw_June_2022.copy()

# Checking the shape
print(f"There are {df_June_2022.shape[0]} rows and {df_June_2022.shape[1]} columns.")

# Checking first 2 rows of the data
df_June_2022.head(2)

There are 145 rows and 6 columns.


,month_year,day,name,info,link,num_references
0,June 2022,8,Mladen Frančić,", 67, Croatian football player and manager (Vrbovec, Podravina, Al-Watani Club).",https://en.wikipedia.org/wiki/Mladen_Fran%C4%8Di%C4%87,1
1,June 2022,6,Valery Ryumin,", 82, Russian cosmonaut (Soyuz 25, Soyuz 32, Soyuz 35).",https://en.wikipedia.org/wiki/Valery_Ryumin,2


<IPython.core.display.Javascript object>

#### Observations:
- The June, 2022 data does not follow the same row order as the previous dataframe, which was in order of day of the month. 
- For continuity, before concatinating the two dataframes, we will sort June, 2022 by day.

In [9]:
# Sorting by day
df_June_2022.sort_values(by="day", inplace=True)

# Re-checking first 2 rows of the data
df_June_2022.head(2)

,month_year,day,name,info,link,num_references
26,June 2022,1,Richard Oldcorn,", 84, English Olympic fencer (1964, 1968, 1972).",https://en.wikipedia.org/wiki/Richard_Oldcorn,5
20,June 2022,1,István Szőke,", 75, Hungarian footballer (Ferencváros, national team), stroke.",https://en.wikipedia.org/wiki/Istv%C3%A1n_Sz%C5%91ke,2


<IPython.core.display.Javascript object>

In [10]:
# Checking last 2 rows of the data
df_June_2022.tail(2)

,month_year,day,name,info,link,num_references
8,June 2022,9,Oleg Moliboga,", 69, Russian volleyball player, Olympic champion (1980) and coach.",https://en.wikipedia.org/wiki/Oleg_Moliboga,2
5,June 2022,9,Zou Jing,", 86, Chinese engineer, member of the Chinese Academy of Engineering.",https://en.wikipedia.org/wiki/Zou_Jing_(engineer),3


<IPython.core.display.Javascript object>

In [11]:
# Checking a sample of the data
df_June_2022.sample(5)

,month_year,day,name,info,link,num_references
50,June 2022,2,Peter Daley,", 71, American politician, member of the Pennsylvania House of Representatives (1983–2016).",https://en.wikipedia.org/wiki/Peter_Daley,8
21,June 2022,1,Mark Schaeffer,", 73, American baseball player (San Diego Padres).",https://en.wikipedia.org/wiki/Mark_Schaeffer,5
66,June 2022,3,Sophie Freud,", 97, Austrian psychologist, pancreatic cancer.",https://en.wikipedia.org/wiki/Sophie_Freud,19
70,June 2022,3,Piergiorgio Bressani,", 92, Italian politician, deputy (1963–1986), and mayor of Udine (1985–1990).",https://en.wikipedia.org/wiki/Piergiorgio_Bressani,3
120,June 2022,6,William J. Sullivan,", 83, American judge, member (1999–2009) and chief justice of the Connecticut Supreme Court (2001–2006).",https://en.wikipedia.org/wiki/William_J._Sullivan,5


<IPython.core.display.Javascript object>

#### Observations:
- Now, we are ready to combine the three dataframes.

<a id='step2'></a>
## Combining Dataframes

### Adding Number of References to 1994 through May 2022 Data

In [12]:
# Adding num_references column to 1994 through May 2022 data
df_combined = pd.merge(df_94_to_22, df_reference_counts, how="left", on="link")

# Checking first 2 rows of the data
df_combined.head(2)

,month_year,day,name,info,link,num_references
0,January 1994,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21
1,January 1994,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12


<IPython.core.display.Javascript object>

### Adding June 2022 Data

In [13]:
# Adding Juned 2022 data
df_combined = pd.concat([df_combined, df_June_2022], ignore_index=True)

# Making a working copy
df = df_combined.copy()

# Checking the shape
print(f"There are {df.shape[0]} rows and {df.shape[1]} columns.")

# Checking first 2 rows of the data
df.head(2)

There are 133914 rows and 6 columns.


,month_year,day,name,info,link,num_references
0,January 1994,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21
1,January 1994,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12


<IPython.core.display.Javascript object>

In [14]:
# Checking last 2 rows of the data
df.tail(2)

,month_year,day,name,info,link,num_references
133912,June 2022,9,Oleg Moliboga,", 69, Russian volleyball player, Olympic champion (1980) and coach.",https://en.wikipedia.org/wiki/Oleg_Moliboga,2
133913,June 2022,9,Zou Jing,", 86, Chinese engineer, member of the Chinese Academy of Engineering.",https://en.wikipedia.org/wiki/Zou_Jing_(engineer),3


<IPython.core.display.Javascript object>

In [15]:
# Checking a sample of the data
df.sample(5)

,month_year,day,name,info,link,num_references
111940,April 2020,14,Maria de Sousa,", 80, Portuguese immunologist, COVID-19.",https://en.wikipedia.org/wiki/Maria_de_Sousa,8
15368,February 2000,14,Jimmy Martin,", 75, Irish professional golfer.",https://en.wikipedia.org/wiki/Jimmy_Martin_(golfer),3
129651,December 2021,27,Earl Best,", 74, American activist and convicted bank robber, cancer.",https://en.wikipedia.org/wiki/Earl_Best,16
81219,February 2016,16,Gustavo Julian Garcia,", 43, American criminal, execution by lethal injection.",https://en.wikipedia.org/wiki/Gustavo_Julian_Garcia,NaN
107223,October 2019,12,Carlo Croccolo,", 92, Italian actor, voice actor, director and screenwriter. (, , )",https://en.wikipedia.org/wiki/Carlo_Croccolo,NaN


<IPython.core.display.Javascript object>

#### Confirming Correct Number of Resultant Entries

In [16]:
# Confirming correct number of total rows
df_94_to_22.shape[0] + df_June_2022.shape[0]

del df_94_to_22, df_reference_counts, df_June_2022

<IPython.core.display.Javascript object>

#### Observations:
- We have successfully combined the three dataframes.
- Now, we can check for data types, duplicates, and missing values.

<a id='step3'></a>
## Checking data types, duplicates, and null values

### Data types

In [17]:
# Checking data types and null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133914 entries, 0 to 133913
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   month_year      133914 non-null  object
 1   day             133914 non-null  object
 2   name            133903 non-null  object
 3   info            133914 non-null  object
 4   link            133914 non-null  object
 5   num_references  120637 non-null  object
dtypes: object(6)
memory usage: 6.1+ MB


<IPython.core.display.Javascript object>

#### Observations:
- There are 6 columns, all of object type.
- `name` and `num_references` both have missing values.
- The data is in a very raw format and there are other columns that have combined information that will need to be extracted.
- For now, we will leave all as object type.

<a id='step3'></a>
### Duplicate Rows

In [18]:
# Checking duplicate rows
df.duplicated().sum()

9

<IPython.core.display.Javascript object>

#### Observations:
- There are 9 duplicate rows that we will drop now.

In [19]:
# Drop duplicate rows
df.drop_duplicates(inplace=True, ignore_index=True)

# Re-check shape
df.shape

(133905, 6)

<IPython.core.display.Javascript object>

<a id='step4'></a>
### Missing Values

In [20]:
# Check percentage of null values by column
df.isnull().sum() / df.count() * 100

month_year        0.000
day               0.000
name              0.008
info              0.000
link              0.000
num_references   11.007
dtype: float64

<IPython.core.display.Javascript object>

In [21]:
# Checking number of missing values per row
df.isnull().sum(axis=1).value_counts()

0    120628
1     13266
2        11
dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- The number of rows missing only 1 value appears consistent with our anticipated missing `num_references`.
- There are 11 rows that are each missing 2 values.  Let us take a closer look at these rows.

In [22]:
# Checking the rows that are missing values for 2 columns
missing_2 = df[df.isnull().sum(axis=1) == 2]
missing_2

,month_year,day,name,info,link,num_references
18937,August 2001,11,None,"Kevin Kowalcyk, 2, known for eating a hamburger contaminated with E. coli O157:H7.",https://en.wikipedia.orgNone,NaN
24985,January 2004,22,None,"Vincent Palmer, 37, British criminal.",https://en.wikipedia.orgNone,NaN
27458,March 2005,1,None,"Barry Stigler, 57, American voice actor.",https://en.wikipedia.orgNone,NaN
34077,July 2007,11,None,"Nana Gualdi, 75, German singer and actress.",https://en.wikipedia.orgNone,NaN
35097,November 2007,11,None,,https://en.wikipedia.orgNone,NaN
41075,May 2009,18,None,Either killed in a missile attack or shot:\n,https://en.wikipedia.orgNone,NaN
64771,September 2013,29,None,"Scott Workman, 47, American stuntman (, , ), cancer.",https://en.wikipedia.orgNone,NaN
76024,April 2015,29,None,Notable convicted drug traffickers executed by Indonesian firing squad:\n,https://en.wikipedia.orgNone,NaN
105871,August 2019,2,None,"Japanese convicted murderers, executed by hanging.\n",https://en.wikipedia.orgNone,NaN
106617,September 2019,12,None,"Thami Shobede, 31, Singer Songwriter",https://en.wikipedia.orgNone,NaN


<IPython.core.display.Javascript object>

#### Observations:
- We can see that multiple rows are missing `name`, but have the name in `info`, so we can extract it later.  
- The missing link itself is not of concern as it serves only as a means by which to retrieve the `num_references` value.
- As there is no associated link for the individual, we can safely replace the NaN `num_references` values for rows with extractable names with 0.
- We can proceed with removing the rows that lack an extractable name, as they also lack other information necessary for the analysis.

In [23]:
# List of rows to keep
keep_rows = [18937, 24985, 27458, 34077, 64771, 106617]

# For loop to replace num_references NaNs with 0 for rows with extractable names
for row in keep_rows:
    df.loc[row, "num_references"] = 0

# List of rows to remove
remove_rows = [index for index in missing_2.index if index not in keep_rows]
del missing_2

# Dropping rows
df.drop(remove_rows, inplace=True)

# Re-checking shape
df.shape

(133900, 6)

<IPython.core.display.Javascript object>

In [24]:
# Re-check info
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 133900 entries, 0 to 133904
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   month_year      133900 non-null  object
 1   day             133900 non-null  object
 2   name            133894 non-null  object
 3   info            133900 non-null  object
 4   link            133900 non-null  object
 5   num_references  120634 non-null  object
dtypes: object(6)
memory usage: 7.2+ MB


<IPython.core.display.Javascript object>

#### Observations:
- There are now only 6 rows with missing `name`, corresponding to the names we identified in `info`, that we will extract later.
- The remaining missing values are all for `num_references`, so we can proceed to make another attempt at scraping this information.
- Let us check a sample of these rows.

In [25]:
# Checking sample of rows missing num_references
df[df["num_references"].isna()].sample(5)

,month_year,day,name,info,link,num_references
48855,February 2011,1,Stanisław Michalski,", 78, Polish actor.",https://en.wikipedia.org/wiki/Stanis%C5%82aw_Michalski,NaN
73352,December 2014,18,Claude Frikart,", 92, French Roman Catholic prelate, Auxiliary Bishop of Paris (1986–1997).",https://en.wikipedia.org/wiki/Claude_Frikart,NaN
66466,January 2014,3,Eric Barnes,", 76, English footballer (Crewe Alexandra).",https://en.wikipedia.org/wiki/Eric_Barnes_(footballer),NaN
70958,August 2014,15,Licia Albanese,", 105, Italian-born American operatic soprano.",https://en.wikipedia.org/wiki/Licia_Albanese,NaN
50736,June 2011,27,Thierry Martens,", 69, Belgian science fiction, detective novels, and comics author.",https://en.wikipedia.org/wiki/Thierry_Martens,NaN


<IPython.core.display.Javascript object>

#### Observations:
- Following the links reveals that the pages contain references.  
- Therefore, they either have a variation in the XPath followed for scraping, or Scrapy had an issue with following their links.
- We will export a dataframe of the links to the pages that need to be re-scraped for `num_references`.

In [26]:
# Exporting dataframe of pages to rescrape for num_references
rescrape_df = df[df["num_references"].isna()]["link"]
rescrape_df.to_csv("rescrape_df.csv", index=False)

del rescrape_df

<IPython.core.display.Javascript object>

<a id='step5'></a>
## Missing `num_references` Values

### First Re-scrape with Spider ["refs2"](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wikipedia_notable_deaths/spiders/refs2.py)

#### Observations:
- A second iteration of scraping individual pages for number of references reveals variation in the XPath for those pages.
- The new spider, ["refs2"](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wikipedia_notable_deaths/spiders/refs2.py), uses an updated XPath.
- As before, we will import and merge the data, examine rows still missing `num_references` and re-scrape them, adjusting the spider's XPath again, if needed.

In [27]:
# Reading the refs2 dataset from sql db and table
conn = sql.connect("refs2.db")
raw_refs2 = pd.read_sql("SELECT * FROM refs2", conn)

# Making a working copy
df_refs2 = raw_refs2.copy()

# Checking the shape
print(f"There are {df_refs2.shape[0]} rows and {df_refs2.shape[1]} columns.")

# Checking first 2 rows of the data
df_refs2.head(2)

There are 7365 rows and 2 columns.


,link,num_references
0,https://en.wikipedia.org/wiki/List_of_American_supercentenarians#Charlotte_Benkner,63
1,https://en.wikipedia.org/wiki/Eugene_Record,11


<IPython.core.display.Javascript object>

In [28]:
# Checking last 2 rows of the data
df_refs2.tail(2)

,link,num_references
7363,https://en.wikipedia.org/wiki/Gunnar_Utterberg,3
7364,https://en.wikipedia.org/wiki/Bill_Sudakis,19


<IPython.core.display.Javascript object>

In [29]:
# Checking a sample of the data
df_refs2.sample(5)

,link,num_references
1995,https://en.wikipedia.org/wiki/Ingeborg_Mello,3
1113,https://en.wikipedia.org/wiki/Michele_Greco,24
1952,https://en.wikipedia.org/wiki/Harry_Bell_(ice_hockey),0
1637,https://en.wikipedia.org/wiki/Susan_Hibbert,1
5218,https://en.wikipedia.org/wiki/Teodoro_Gonz%C3%A1lez_de_Le%C3%B3n,9


<IPython.core.display.Javascript object>

#### Observations:
- We were able to obtain 7365 of the missing values.

#### Adding Missing References to Dataframe

In [30]:
# Adding new num_references column to data
df = pd.merge(df, df_refs2, how="left", on="link")
del df_refs2

# Checking sample of the data
df.sample(5)

,month_year,day,name,info,link,num_references_x,num_references_y
72467,November 2014,2,Dick Breaden,", 69-70, Australian rugby league footballer.",https://en.wikipedia.org/wiki/Dick_Breaden,4,NaN
67720,March 2014,1,Werner Uebelmann,", 92, Swiss entrepreneur and writer.",https://en.wikipedia.org/wiki/Werner_Uebelmann,2,NaN
22810,February 2003,10,Edgar de Evia,", 92, American photographer born in Mérida, Yucatán.",https://en.wikipedia.org/wiki/Edgar_de_Evia,19,NaN
87136,January 2017,4,Lois Rice,", 83, American business executive.",https://en.wikipedia.org/wiki/Lois_Rice,9,NaN
125533,July 2021,27,Jan Pęczek,", 70, Polish actor (), bone marrow cancer.",https://en.wikipedia.org/wiki/Jan_P%C4%99czek,5,NaN


<IPython.core.display.Javascript object>

In [31]:
# Filling missing values with newly obtained values
df["num_references_x"].fillna(df["num_references_y"], inplace=True)
df.sample(5)

,month_year,day,name,info,link,num_references_x,num_references_y
55564,May 2012,9,Constantin Piron,", 80, Belgian physicist.",https://en.wikipedia.org/wiki/Constantin_Piron,4,NaN
24734,December 2003,14,Blas Ople,", 75, Filipino journalist and politician.",https://en.wikipedia.org/wiki/Blas_Ople,22,NaN
73514,December 2014,25,Bernard Kay,", 86, British actor (, , ).",https://en.wikipedia.org/wiki/Bernard_Kay,5,NaN
96546,April 2018,25,Edith MacArthur,", 92, Scottish actress ().",https://en.wikipedia.org/wiki/Edith_MacArthur,13,NaN
121074,February 2021,28,Yousuf Shaaban,", 89, Egyptian actor (, , ), COVID-19.",https://en.wikipedia.org/wiki/Yousuf_Shaaban_(actor),12,NaN


<IPython.core.display.Javascript object>

In [32]:
# Dropping new references column and reverting to original column name
df.drop("num_references_y", axis=1, inplace=True)
df.rename(columns={"num_references_x": "num_references"}, inplace=True)
df.head(2)

,month_year,day,name,info,link,num_references
0,January 1994,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21
1,January 1994,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12


<IPython.core.display.Javascript object>

#### Checking Remaining Missing Values

In [33]:
# Checking remaining missing values
df.isna().sum()

month_year           0
day                  0
name                 6
info                 0
link                 0
num_references    5895
dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- We have nearly 6000 remaining missing values for `num_references`, so we will iterate through the rescraping again.

In [34]:
# Checking sample of rows missing num_references
df[df["num_references"].isna()].sample(5)

,month_year,day,name,info,link,num_references
60496,February 2013,9,Mike Banks,", 90, British mountaineer and Royal Marines officer.",https://en.wikipedia.org/wiki/Mike_Banks_(mountaineer),NaN
115917,September 2020,7,Aurelio Iragorri Hormaza,", 83, Colombian politician, Senator (1991–2014), Governor of Cauca (1975–1976) and President of the Chamber of Representatives (1981–1982), COVID-19.",https://en.wikipedia.org/wiki/Aurelio_Iragorri_Hormaza,NaN
95581,March 2018,9,Harold Rosewarne,", 87, Australian footballer (South Melbourne).",https://en.wikipedia.org/wiki/Harold_Rosewarne,NaN
61258,March 2013,19,Desmond Drummer,", 72, South African cricketer.",https://en.wikipedia.org/wiki/Desmond_Drummer,NaN
47495,October 2010,14,Hermann Scheer,", 66, German politician, member of the Bundestag (1980–2010) and Right Livelihood Award laureate (1999), after short illness.",https://en.wikipedia.org/wiki/Hermann_Scheer,NaN


<IPython.core.display.Javascript object>

#### Observations:
- Following the links again reveals that the pages contain references.  
- We will export another dataframe of the links to the pages that need to be re-scraped for `num_references` and examine the pages for alternate XPaths for scraping.

In [35]:
# Exporting dataframe of pages to rescrape for num_references
rescrape_df_2nd = df[df["num_references"].isna()]["link"]
rescrape_df_2nd.to_csv("rescrape_df_2nd.csv", index=False)
del rescrape_df_2nd

<IPython.core.display.Javascript object>

### Second Re-scrape with Spider ["refs3"](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wikipedia_notable_deaths/spiders/refs3.py)

#### Observations:
- The XPath matched that of the last scraping iteration for several pages, so the scraping was repeated for the remaining rows with missing `num_references`.

In [36]:
# Reading the refs3 dataset from sql db and table
conn = sql.connect("refs3.db")
raw_refs3 = pd.read_sql("SELECT * FROM refs3", conn)

# Making a working copy
df_refs3 = raw_refs3.copy()

# Checking the shape
print(f"There are {df_refs3.shape[0]} rows and {df_refs3.shape[1]} columns.")

# Checking first 2 rows of the data
df_refs3.head(2)

There are 3633 rows and 2 columns.


,link,num_references
0,https://en.wikipedia.org/wiki/List_of_American_supercentenarians#Grace_Thaxton,63
1,"https://en.wikipedia.org/wiki/Christopher_Prout,_Baron_Kingsland",6


<IPython.core.display.Javascript object>

In [37]:
# Checking last 2 rows of the data
df_refs3.tail(2)

,link,num_references
3631,https://en.wikipedia.org/wiki/Concepci%C3%B3n_Ram%C3%ADrez,8
3632,https://en.wikipedia.org/wiki/Mary_Mahoney_(physician),6


<IPython.core.display.Javascript object>

In [38]:
# Checking a sample of the data
df_refs3.sample(5)

,link,num_references
371,https://en.wikipedia.org/wiki/Harry_Brown_Bainbridge_III,0
757,https://en.wikipedia.org/wiki/Jack_Jones_(journalist),4
1356,https://en.wikipedia.org/wiki/Kazimiera_Rykowska,2
3219,https://en.wikipedia.org/wiki/Ding_Xieping,2
737,https://en.wikipedia.org/wiki/Asker_Dzhappuyev,5


<IPython.core.display.Javascript object>

#### Observations:
- We were able to obtain 3633 of the missing values.

#### Adding Missing References to Dataframe

In [39]:
# Adding new num_references column to data
df = pd.merge(df, df_refs3, how="left", on="link")
del df_refs3

# Checking sample of the data
df.sample(5)

,month_year,day,name,info,link,num_references_x,num_references_y
19295,September 2001,28,Jack Maguire,", 76, American baseball player.",https://en.wikipedia.org/wiki/Jack_Maguire_(baseball),1,NaN
101562,January 2019,15,Gabriel Montcharmont,", 78, French politician, Deputy (1988–1993, 1997–2002).",https://en.wikipedia.org/wiki/Gabriel_Montcharmont,1,NaN
90583,June 2017,24,Monica Nordquist,", 76, Swedish actress ().",https://en.wikipedia.org/wiki/Monica_Nordquist,3,NaN
7762,January 1997,15,Ahmad Tafazzoli,", 59, Iranian Iranist and professor of ancient Iranian languages, homicide.",https://en.wikipedia.org/wiki/Ahmad_Tafazzoli,3,NaN
114641,July 2020,20,Günter-Helge Strickstrack,", 99, German politician, founding member of the CDU.",https://en.wikipedia.org/wiki/G%C3%BCnter-Helge_Strickstrack,2,NaN


<IPython.core.display.Javascript object>

In [40]:
# Filling missing values with newly obtained values
df["num_references_x"].fillna(df["num_references_y"], inplace=True)
df.sample(5)

,month_year,day,name,info,link,num_references_x,num_references_y
3792,July 1995,4,Karim Sanjabi,", 89, Iranian politician.",https://en.wikipedia.org/wiki/Karim_Sanjabi,11,NaN
85746,October 2016,20,Uwe Dreher,", 56, German footballer (Stuttgarter Kickers).",https://en.wikipedia.org/wiki/Uwe_Dreher,2,2
101586,January 2019,16,Aya Maasarwe,", 21, Israeli exchange student, homicide.",https://en.wikipedia.org/wiki/Aya_Maasarwe,19,NaN
78270,September 2015,8,Carlo Schäfer,", 51, German author.",https://en.wikipedia.org/wiki/Carlo_Sch%C3%A4fer,1,NaN
76043,May 2015,1,Amitabha Chowdhury,", 87, Indian journalist.",https://en.wikipedia.org/wiki/Amitabha_Chowdhury,2,NaN


<IPython.core.display.Javascript object>

In [41]:
# Dropping new references column and reverting to original column name
df.drop("num_references_y", axis=1, inplace=True)
df.rename(columns={"num_references_x": "num_references"}, inplace=True)
df.head(2)

,month_year,day,name,info,link,num_references
0,January 1994,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21
1,January 1994,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12


<IPython.core.display.Javascript object>

#### Checking Remaining Missing Values

In [42]:
# Checking remaining missing values
df.isna().sum()

month_year           0
day                  0
name                 6
info                 0
link                 0
num_references    2260
dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- We have 2260 remaining missing values for `num_references`, so we will iterate through the rescraping again.

In [43]:
# Checking sample of rows missing num_references
df[df["num_references"].isna()].sample(5)

,month_year,day,name,info,link,num_references
36198,February 2008,19,Samuel Champkin,", 28, American singer in Metal band Tech Giants, car crash.",https://en.wikipedia.org/w/index.php?title=Samuel_Champkin&action=edit&redlink=1,NaN
102836,March 2019,10,Eric Moss,", 44, American football player (Minnesota Vikings, Scottish Claymores).",https://en.wikipedia.org/wiki/Eric_Moss,NaN
70740,August 2014,4,Irma Elizondo Ramírez,", 68, Mexican politician, MP for Coahuila (since 2012), heart attack.",https://en.wikipedia.org/wiki/Irma_Elizondo_Ram%C3%ADrez,NaN
104606,June 2019,3,Evangelina García Prince,", 84, Venezuelan women's rights activist, politician and academic, Senator (1988–1991, 1994–1996) and member of the Comisión para la Reforma del E...",https://en.wikipedia.org/wiki/Evangelina_Garc%C3%ADa_Prince,NaN
107862,November 2019,16,Irma Cordero,", 77, Peruvian Olympic volleyball player (1968, 1976).",https://en.wikipedia.org/wiki/Irma_Cordero,NaN


<IPython.core.display.Javascript object>

#### Observations:
- Following the links again reveals that the pages contain references.  
- We will export another dataframe of the links to the pages that need to be re-scraped for `num_references` and examine the pages for alternate XPaths for scraping.

In [44]:
# Exporting dataframe of pages to rescrape for num_references
rescrape_df_3rd = df[df["num_references"].isna()]["link"]
rescrape_df_3rd.to_csv("rescrape_df_3rd.csv", index=False)
del rescrape_df_3rd

<IPython.core.display.Javascript object>

### Third Re-scrape with Spider ["refs4"](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wikipedia_notable_deaths/spiders/refs4.py)

#### Observations:
- The XPath matched that of the last scraping iteration for several pages, so the scraping was repeated for the remaining rows with missing `num_references`.

In [45]:
# Reading the refs4 dataset from sql db and table
conn = sql.connect("refs4.db")
raw_refs4 = pd.read_sql("SELECT * FROM refs4", conn)

# Making a working copy
df_refs4 = raw_refs4.copy()

# Checking the shape
print(f"There are {df_refs4.shape[0]} rows and {df_refs4.shape[1]} columns.")

# Checking first 2 rows of the data
df_refs4.head(2)

There are 1960 rows and 2 columns.


,link,num_references
0,https://en.wikipedia.org/wiki/List_of_French_supercentenarians,157
1,https://en.wikipedia.org/wiki/Bob_Kahler,3


<IPython.core.display.Javascript object>

In [46]:
# Checking last 2 rows of the data
df_refs4.tail(2)

,link,num_references
1958,https://en.wikipedia.org/wiki/Jorge_Sampaio,73
1959,https://en.wikipedia.org/wiki/Irma_Kalish,14


<IPython.core.display.Javascript object>

In [47]:
# Checking a sample of the data
df_refs4.sample(5)

,link,num_references
497,https://en.wikipedia.org/wiki/Iain_Hesford,6
1128,https://en.wikipedia.org/wiki/Ian_Mason_(cricketer),3
1493,https://en.wikipedia.org/wiki/Toivo_Topias_Pohjala,2
347,https://en.wikipedia.org/wiki/John_M._Fitzpatrick,8
1281,https://en.wikipedia.org/wiki/Niranjan_Bhagat,7


<IPython.core.display.Javascript object>

#### Observations:
- We were able to obtain 1960 of the missing values.

#### Adding Missing References to Dataframe

In [48]:
# Adding new num_references column to data
df = pd.merge(df, df_refs4, how="left", on="link")
del df_refs4

# Checking sample of the data
df.sample(5)

,month_year,day,name,info,link,num_references_x,num_references_y
124153,June 2021,6,John Ernest Ekuban,", 84, Ghanaian politician, MP (1993–1997).",https://en.wikipedia.org/wiki/John_Ernest_Ekuban,3,NaN
39160,December 2008,5,Constantin Ticu Dumitrescu,", 80, Romanian politician, cancer.",https://en.wikipedia.org/wiki/Constantin_Ticu_Dumitrescu,3,NaN
121201,March 2021,4,Hugh Newell Jacobsen,", 91, American architect, complications from COVID-19.",https://en.wikipedia.org/wiki/Hugh_Newell_Jacobsen,11,NaN
97679,June 2018,23,Koro Wētere,", 83, New Zealand politician, MP for Western Maori (1969–1996), Minister of Māori Affairs (1984–1990).",https://en.wikipedia.org/wiki/Koro_W%C4%93tere,13,NaN
131814,March 2022,5,Colin Wesley,", 84, South African cricketer (national team), complications from a stroke.",https://en.wikipedia.org/wiki/Colin_Wesley,6,NaN


<IPython.core.display.Javascript object>

In [49]:
# Filling missing values with newly obtained values
df["num_references_x"].fillna(df["num_references_y"], inplace=True)
df.sample(5)

,month_year,day,name,info,link,num_references_x,num_references_y
29146,October 2005,20,Saadoun Sughaiyer al-Janabi,", Iraqi defense lawyer in Saddam Hussein's trial, murdered by unknown assailants in Baghdad.",https://en.wikipedia.org/wiki/Saadoun_Sughaiyer_al-Janabi,6,NaN
53821,February 2012,1,John Harrison,", 87, Australian Olympic rower.",https://en.wikipedia.org/wiki/John_Harrison_(rower),3,NaN
131928,March 2022,10,Jürgen Grabowski,", 77, German footballer (Eintracht Frankfurt, West Germany national team).",https://en.wikipedia.org/wiki/J%C3%BCrgen_Grabowski,21,NaN
48833,January 2011,31,Bartolomeu Anania,", 89, Romanian Orthodox Metropolitan of Cluj-Napoca, Alba Iulia, Crişana and Maramureş (since 1993).",https://en.wikipedia.org/wiki/Bartolomeu_Anania,18,NaN
112301,April 2020,25,Devanand Konwar,", 77, Indian politician, Governor of West Bengal (2009–2010), Bihar (2009–2013) and Tripura (2013–2014).",https://en.wikipedia.org/wiki/Devanand_Konwar,1,NaN


<IPython.core.display.Javascript object>

In [50]:
# Dropping new references column and reverting to original column name
df.drop("num_references_y", axis=1, inplace=True)
df.rename(columns={"num_references_x": "num_references"}, inplace=True)
df.head(2)

,month_year,day,name,info,link,num_references
0,January 1994,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21
1,January 1994,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12


<IPython.core.display.Javascript object>

#### Checking Remaining Missing Values

In [51]:
# Checking remaining missing values
df.isna().sum()

month_year          0
day                 0
name                6
info                0
link                0
num_references    300
dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- We have just 300 remaining missing values for `num_references`, so we will iterate through the rescraping again.

In [52]:
# Checking sample of rows missing num_references
df[df["num_references"].isna()].sample(5)

,month_year,day,name,info,link,num_references
126305,August 2021,22,Karl Traub,", 80, German politician, member of the Landtag of Baden-Württemberg (1996–2016), heart attack.",https://en.wikipedia.org/wiki/Karl_Traub,NaN
120609,February 2021,15,Andreas Apostolopoulos,", 69, Greek-Canadian real estate developer.",https://en.wikipedia.org/wiki/Andreas_Apostolopoulos,NaN
118214,December 2020,10,Cal Hockley,", 85, Canadian ice hockey player (Trail Smoke Eaters).",https://en.wikipedia.org/wiki/Cal_Hockley,NaN
24346,October 2003,15,Ray Kuhlman,", 84, American pilot and businessman.",https://en.wikipedia.org/w/index.php?title=Ray_Kuhlman&action=edit&redlink=1,NaN
116313,September 2020,24,Bertram Richardson,", 88, English cricketer (Derbyshire).",https://en.wikipedia.org/wiki/Bertram_Richardson,NaN


<IPython.core.display.Javascript object>

#### Observations:
- Following the links again reveals that the pages contain references.  
- We will export another dataframe of the links to the pages that need to be re-scraped for `num_references` and examine the pages for alternate XPaths for scraping.

In [53]:
# Exporting dataframe of pages to rescrape for num_references
rescrape_df_4th = df[df["num_references"].isna()]["link"]
rescrape_df_4th.to_csv("rescrape_df_4th.csv", index=False)
del rescrape_df_4th

<IPython.core.display.Javascript object>

### Fourth Re-scrape with Spider ["refs5"](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wikipedia_notable_deaths/spiders/refs5.py)

#### Observations:
- The XPath matched that of the last scraping iteration for several pages, so the scraping was repeated for the remaining rows with missing `num_references`.

In [54]:
# Reading the refs5 dataset from sql db and table
conn = sql.connect("refs5.db")
raw_refs5 = pd.read_sql("SELECT * FROM refs5", conn)

# Making a working copy
df_refs5 = raw_refs5.copy()

# Checking the shape
print(f"There are {df_refs5.shape[0]} rows and {df_refs5.shape[1]} columns.")

# Checking first 2 rows of the data
df_refs5.head(2)

There are 208 rows and 2 columns.


,link,num_references
0,https://en.wikipedia.org/wiki/Mohamed_Haytham_Khayat,5
1,https://en.wikipedia.org/wiki/Sid_McCray,6


<IPython.core.display.Javascript object>

In [55]:
# Checking last 2 rows of the data
df_refs5.tail(2)

,link,num_references
206,https://en.wikipedia.org/wiki/Anna_Cataldi,9
207,https://en.wikipedia.org/wiki/Allan_Egolf,4


<IPython.core.display.Javascript object>

In [56]:
# Checking a sample of the data
df_refs5.sample(5)

,link,num_references
77,https://en.wikipedia.org/wiki/Gildardo_Garc%C3%ADa,9
34,https://en.wikipedia.org/wiki/Gholam-Abbas_Tavassoli,4
148,https://en.wikipedia.org/wiki/G%C3%A9rard_Vergnaud,1
112,https://en.wikipedia.org/wiki/Mack_Walker,3
150,https://en.wikipedia.org/wiki/Paula_Caplan,5


<IPython.core.display.Javascript object>

#### Observations:
- We were able to obtain 208 of the missing values.

#### Adding Missing References to Dataframe

In [57]:
# Adding new num_references column to data
df = pd.merge(df, df_refs5, how="left", on="link")
del df_refs5

# Checking sample of the data
df.sample(5)

,month_year,day,name,info,link,num_references_x,num_references_y
74448,February 2015,8,Rauni-Leena Luukanen-Kilde,", 75, Finnish parapsychologist.",https://en.wikipedia.org/wiki/Rauni-Leena_Luukanen-Kilde,11,NaN
70003,June 2014,25,Ivan Plyushch,", 72, Ukrainian politician, Chairman of the Verkhovna Rada (1990, 1991–1994, 2000–2002), cancer.",https://en.wikipedia.org/wiki/Ivan_Plyushch,9,NaN
125956,August 2021,10,Pio Teek,", 74, Namibian judge, justice of the Supreme Court (2003–2005).",https://en.wikipedia.org/wiki/Pio_Teek,5,NaN
106248,August 2019,24,Tony Nichols,", 81, Australian Anglican prelate, Bishop of North West Australia (1992–2003).",https://en.wikipedia.org/wiki/Tony_Nichols,5,NaN
122750,April 2021,21,Marc Bécam,", 89, French politician, deputy (1967–1978), senator (1980–1986), mayor of Quimper (1977–1989).",https://en.wikipedia.org/wiki/Marc_B%C3%A9cam,4,NaN


<IPython.core.display.Javascript object>

In [58]:
# Filling missing values with newly obtained values
df["num_references_x"].fillna(df["num_references_y"], inplace=True)
df.sample(5)

,month_year,day,name,info,link,num_references_x,num_references_y
14986,December 1999,22,Howard H. Cooksey,", 78, United States Army Lieutenant General, heart disease.",https://en.wikipedia.org/wiki/Howard_H._Cooksey,3,NaN
63476,July 2013,16,Mario Laserna Pinzón,", 89, French-born Colombian educator, diplomat and politician, Ambassador to France and Austria, Senator (1991–1995), Alzheimer's disease.",https://en.wikipedia.org/wiki/Mario_Laserna_Pinz%C3%B3n,11,NaN
122528,April 2021,15,Moshe Ber Beck,", 86, Hungarian-born American rabbi and anti-Zionist campaigner, leader of the Neturei Karta, complications of COVID-19.",https://en.wikipedia.org/wiki/Moshe_Ber_Beck,4,NaN
127923,October 2021,23,Abdelbaki Hermassi,", 83, Tunisian politician, minister of foreign affairs (2004–2005).",https://en.wikipedia.org/wiki/Abdelbaki_Hermassi,4,NaN
89976,May 2017,21,Wayne Simoneau,", 82, American politician.",https://en.wikipedia.org/wiki/Wayne_Simoneau,2,NaN


<IPython.core.display.Javascript object>

In [59]:
# Dropping new references column and reverting to original column name
df.drop("num_references_y", axis=1, inplace=True)
df.rename(columns={"num_references_x": "num_references"}, inplace=True)
df.head(2)

,month_year,day,name,info,link,num_references
0,January 1994,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21
1,January 1994,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12


<IPython.core.display.Javascript object>

#### Checking Remaining Missing Values

In [60]:
# Checking remaining missing values
df.isna().sum()

month_year         0
day                0
name               6
info               0
link               0
num_references    92
dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- We have just 92 remaining missing values for `num_references`.
- Let us examine the remaining rows with missing values more closely.

In [61]:
# Checking sample of rows missing num_references
df[df["num_references"].isna()]

,month_year,day,name,info,link,num_references
12476,October 1998,29,Sjoerdtsje Faber,", 83, Dutch speed skater.",https://en.wikipedia.org/w/index.php?title=Sjoerdje_Faber&action=edit&redlink=1,NaN
15859,May 2000,10,Bill Foster,", 68, American entertainer.",https://en.wikipedia.org/w/index.php?title=Bill_Foster_(performer)&action=edit&redlink=1,NaN
18561,June 2001,20,Demetreus Nix,"Douglas Scott, 20, High-school student murdered by .",https://en.wikipedia.org/w/index.php?title=Demetreus_Nix&action=edit&redlink=1,NaN
19905,December 2001,18,Sietske Pasveer,", 86, Dutch speed skater.",https://en.wikipedia.org/w/index.php?title=Sietske_Pasveer&action=edit&redlink=1,NaN
22942,March 2003,3,Gilbert Wheeler Beebe,", 90, American epidemiologist and statistician, conducted ground-breaking radiation exposure studies.",https://en.wikipedia.org/w/index.php?title=Gilbert_Wheeler_Beebe&action=edit&redlink=1,NaN
23291,April 2003,27,Charles A. Marvin,", 73, American district attorney and judge.",https://en.wikipedia.org/w/index.php?title=Charles_A._Marvin&action=edit&redlink=1,NaN
24346,October 2003,15,Ray Kuhlman,", 84, American pilot and businessman.",https://en.wikipedia.org/w/index.php?title=Ray_Kuhlman&action=edit&redlink=1,NaN
33653,May 2007,20,Moses Siregar,", 96, Maritime Chef and American World War 2 Veteran",https://en.wikipedia.org/w/index.php?title=Moses_Siregar&action=edit&redlink=1,NaN
35963,January 2008,31,Michael A. Dions,", 90, American Olympic swimmer, natural causes",https://en.wikipedia.org/w/index.php?title=Michael_A._Dions&action=edit&redlink=1,NaN
36198,February 2008,19,Samuel Champkin,", 28, American singer in Metal band Tech Giants, car crash.",https://en.wikipedia.org/w/index.php?title=Samuel_Champkin&action=edit&redlink=1,NaN


<IPython.core.display.Javascript object>

#### Observations:
- None of the remaining links direct to a personal page for the individual, as such a page does not yet exist.
- As such, as the rows contain the other elements necessary for analysis, it is safe to replace the missing `num_references` values with 0, for these rows.

In [62]:
# Fill remaining missing values for num_references with 0
df["num_references"].fillna(0, inplace=True)

# Recheck missing values
df.isna().sum()

month_year        0
day               0
name              6
info              0
link              0
num_references    0
dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- All of the missing values for `num_references` have been addressed.
- The 6 remaining missing values for `name` will be fixed during data cleaning.
- We now have our complete raw dataset, which we will also export to a SQLite database.
- Then it is time to start cleaning the data.

### [Exporting Dataset to SQLite Database wp_notable_deaths_raw_complete.db](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wp_notable_deaths_raw_complete.db)

In [63]:
# Saving complete raw dataset in a SQLite database
conn = sql.connect("wp_notable_deaths_raw_complete.db")
df.to_sql("wp_notable_deaths_raw_complete", conn)

133900

<IPython.core.display.Javascript object>

# [Proceed to Notebook 2 of  4:  Data Cleaning]